In [1]:
import pandas as pd
import numpy as np

In [141]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

c:\encoder\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import datasets
from datasets import load_dataset

In [4]:
import pandas as pd

splits = {'train': 'train_df.csv', 'validation': 'val_df.csv', 'test': 'test_df.csv'}
train = pd.read_csv("hf://datasets/Sp1786/multiclass-sentiment-analysis-dataset/" + splits["train"])

In [ ]:
test = pd.read_csv("hf://datasets/Sp1786/multiclass-sentiment-analysis-dataset/" + splits["test"])

In [6]:
train.shape

(31232, 4)

In [7]:
train

,id,text,label,sentiment
0,9536,"Cooking microwave pizzas, yummy",2,positive
1,6135,Any plans of allowing sub tasks to show up in ...,1,neutral
2,17697,"I love the humor, I just reworded it. Like sa...",2,positive
3,14182,naw idk what ur talkin about,1,neutral
4,17840,That sucks to hear. I hate days like that,0,negative
...,...,...,...,...
31227,6265,Grrrr....I got the wrong size coat for the sheep,0,negative
31228,11284,4 cases of swine flu!,1,neutral
31229,6436,excellent,2,positive
31230,860,is sitting thru the boring bits in Titanic wai...,1,neutral


In [11]:
test

,id,text,label,sentiment
0,9235,getting cds ready for tour,1,neutral
1,16790,"MC, happy mother`s day to your mom ;).. love yah",2,positive
2,24840,A year from now is graduation....i am pretty s...,0,negative
3,20744,because you had chips and sale w/o me,1,neutral
4,6414,Great for organising my work life balance,2,positive
...,...,...,...,...
5201,10277,Fuck no internet damn time warner!,0,negative
5202,8610,Looking forward to android 1.5 being pushed t...,1,neutral
5203,8114,Not good. Wasted time.,0,negative
5204,3034,"U were great, as always. But, can`t we do an ...",2,positive


In [133]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


tokenizer.pad_token = tokenizer.pad_token or tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def formatting(df):
    
    prompts = [q for q in df["text"]]

    tokenized_inputs = tokenizer(prompts, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

    return {
        "input_ids": tokenized_inputs["input_ids"],
        "attention_mask": tokenized_inputs["attention_mask"],
    }


formatted_train = formatting(train)

In [137]:
labels=train['label']

In [ ]:
class TransformerDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

In [136]:
train_dataset = TransformerDataset(
    formatted_train['input_ids'], 
    formatted_train['attention_mask'], 
    labels
)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [139]:
formatted_train

{'input_ids': tensor([[  101,  8434, 18302,  ...,     0,     0,     0],
         [  101,  2151,  3488,  ...,     0,     0,     0],
         [  101,  1045,  2293,  ...,     0,     0,     0],
         ...,
         [  101,  6581,   102,  ...,     0,     0,     0],
         [  101,  2003,  3564,  ...,     0,     0,     0],
         [  101,  4771,  1996,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [10]:
formatted_train['input_ids'][0]

tensor([  101,  8434, 18302, 10733,  2015,  1010,  9805, 18879,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

![alt text](image.png)

In [40]:
def PositionalEncoding():
    m=torch.zeros(512,512, dtype=torch.float)
    d_model=512
    position=torch.arange(0,512,dtype=torch.float).unsqueeze(1)
    div=torch.exp(torch.arange(0,512,2).float() * (-np.log(10000.0) / d_model))
    m[:,0::2]=torch.sin(position*div)
    m[:,1::2]=torch.cos(position*div)
    return m

In [60]:
def input_final(embedding,attention_mask):
    m=PositionalEncoding()
    return embedding+attention_mask*PositionalEncoding()


In [66]:
import math

In [94]:
class SelfAttention(nn.Module):
    def __init__(self,d_model):
        super().__init__()
        self.d_model=d_model
        self.wq=nn.Linear(d_model,d_model)
        self.wk=nn.Linear(d_model,d_model)
        self.wv=nn.Linear(d_model,d_model)
    def forward(self,x):
        query=self.wq(x)
        key=self.wk(x)
        value=self.wv(x)
        scores=torch.matmul(query,key.transpose(-2,-1)) / math.sqrt(self.d_model)
        scores=torch.softmax(scores,dim=-1)
        output=torch.matmul(scores,value)
        return output

In [110]:
class Classifier(nn.Module):
    def __init__(self,d_model,num_classes):
        super().__init__()
        self.fc=nn.Sequential(
            nn.Linear(d_model,256),
            nn.ReLU(),
            nn.Linear(256,num_classes)
        )
    def forward(self,x):
        x=torch.mean(x,dim=1)
        return self.fc(x)

In [38]:
position=torch.arange(0,512,dtype=torch.float).unsqueeze(1)
print(position.shape)

torch.Size([512, 1])


In [99]:
class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=512, num_classes=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.attn = SelfAttention(d_model)
        self.classifier = Classifier(d_model, num_classes)

    def forward(self, input_ids, attention_mask):
        x = self.embedding(input_ids)                             
        x = input_final(x, attention_mask.unsqueeze(-1))        
        x = self.attn(x)                                          
        logits = self.classifier(x)                            
        return logits

In [108]:
import torch.optim as optim


In [118]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleTransformer(vocab_size=tokenizer.vocab_size, num_classes=3).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

epochs = 2
for epoch in range(epochs):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    print(f"\nEpoch {epoch+1}/{epochs}")
    for step, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)  # shape: [batch_size, num_classes]
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # Predictions and accuracy
        preds = torch.argmax(outputs, dim=1)
        correct = (preds == targets).sum().item()
        total = targets.size(0)

        # Logging
        accuracy = correct / total
        print(f"  Step {step+1}: Loss = {loss.item():.4f}, Accuracy = {accuracy:.4f}")

        # Accumulate totals
        total_loss += loss.item()
        total_correct += correct
        total_samples += total

    epoch_accuracy = total_correct / total_samples
    print(f"Epoch {epoch+1} Summary → Loss: {total_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")




Epoch 1/2
  Step 1: Loss = 1.1261, Accuracy = 0.1250
  Step 2: Loss = 1.0775, Accuracy = 0.5000
  Step 3: Loss = 1.0834, Accuracy = 0.5000
  Step 4: Loss = 1.0786, Accuracy = 0.5000
  Step 5: Loss = 1.2225, Accuracy = 0.2500
  Step 6: Loss = 1.2504, Accuracy = 0.2500
  Step 7: Loss = 1.1118, Accuracy = 0.3750
  Step 8: Loss = 1.1369, Accuracy = 0.3750
  Step 9: Loss = 1.1315, Accuracy = 0.2500
  Step 10: Loss = 0.9553, Accuracy = 0.6250
  Step 11: Loss = 1.0854, Accuracy = 0.2500
  Step 12: Loss = 1.1454, Accuracy = 0.1250
  Step 13: Loss = 1.1891, Accuracy = 0.5000
  Step 14: Loss = 1.0401, Accuracy = 0.3750
  Step 15: Loss = 1.1058, Accuracy = 0.2500
  Step 16: Loss = 1.1406, Accuracy = 0.2500
  Step 17: Loss = 0.9912, Accuracy = 0.6250
  Step 18: Loss = 1.1119, Accuracy = 0.2500
  Step 19: Loss = 1.2030, Accuracy = 0.0000
  Step 20: Loss = 1.0329, Accuracy = 0.6250
  Step 21: Loss = 1.1104, Accuracy = 0.2500
  Step 22: Loss = 1.0842, Accuracy = 0.5000
  Step 23: Loss = 1.1485, Accu

Inference

In [ ]:

tokenizer.pad_token = tokenizer.pad_token or tokenizer.add_special_tokens({'pad_token': '[PAD]'})


test_text = "i went to my friend's house and we had a fight but we made up later and now we are good friends again"


encoding = tokenizer(test_text, 
                     padding='max_length', 
                     truncation=True, 
                     max_length=512, 
                     return_tensors='pt')


input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)


model.eval()
with torch.no_grad():
    output = model(input_ids, attention_mask)
    predicted_class = torch.argmax(output, dim=1).item()

print(f"Input Text: {test_text}")
print(f"Predicted class: {predicted_class}")


Input Text: i went to my friend's house and we had a fight but we made up later and now we are good friends again
Predicted class: 2


In [146]:
train

,id,text,label,sentiment
0,9536,"Cooking microwave pizzas, yummy",2,positive
1,6135,Any plans of allowing sub tasks to show up in ...,1,neutral
2,17697,"I love the humor, I just reworded it. Like sa...",2,positive
3,14182,naw idk what ur talkin about,1,neutral
4,17840,That sucks to hear. I hate days like that,0,negative
...,...,...,...,...
31227,6265,Grrrr....I got the wrong size coat for the sheep,0,negative
31228,11284,4 cases of swine flu!,1,neutral
31229,6436,excellent,2,positive
31230,860,is sitting thru the boring bits in Titanic wai...,1,neutral
